In [1]:

import copy
import sys
from typing import Dict, List, Tuple
from pandas import DataFrame, to_datetime, Series

from freqtrade import (DependencyException, OperationalException, exchange, persistence)
from freqtrade.arguments import Arguments
from freqtrade.configuration import Configuration
from freqtrade.analyze import Analyze
from freqtrade.exchange import get_ticker_history
from freqtrade.logger import Logger
from freqtrade.persistence import Trade, Pair
from freqtrade.optimize import load_tickerdata_file, download_backtesting_testdata



from scipy import linspace, polyval, polyfit, sqrt, stats, randn
from pylab import plot, title, show , legend
from freqtrade.indicators import in_range
# import operator

import numpy as np
import matplotlib.pyplot as plt

# ZigZag

# This is inside your IPython Notebook
import pyximport
pyximport.install(reload_support=True)
from freqtrade.vendor.zigzag_hi_lo import *
# from zigzag import *


pair = "BTC/USDT"
# pair = "DCR/BTC"
timeframes = ['1d', '1h', '1m']

# timeframe = 1000

sysargv=['-s', 'trend001', '-c', '../config_testing.json']

arguments = Arguments(
    sysargv,
    'Simple High Frequency Trading Bot for crypto currencies'
)
args = arguments.get_parsed_arg()

# Initialize all modules
configuration = Configuration(args)
config = configuration.get_config()
analyze = Analyze(config)

logger = Logger(name=__name__, level=config.get('loglevel')).get_logger()

persistence.init(config)
exchange.init(config)

interval = analyze.get_ticker_interval()


# download_backtesting_testdata('../freqtrade/tests/testdata/', 'BTC/USDT', '1m')

def get_df(pair: str, interval: str) -> DataFrame:
    """
    Calculates current signal based several technical analysis indicators
    :param pair: pair in format ANT/BTC
    :param interval: Interval to use (in min)
    :return: (Buy, Sell) A bool-tuple indicating buy/sell signal
    """
    print('interval : ',interval)
    
    
    ticker_hist = get_ticker_history(pair, interval)
#     ticker_hist = load_tickerdata_file('../freqtrade/tests/testdata_full/', pair, interval)
#     print (ticker_hist)
#     print (ticker_hist)
    if not ticker_hist:
        logger.warning('Empty ticker history for pair %s', pair)
        return None

    try:
        dataframe = analyze.analyze_ticker(ticker_hist, pair, interval)
    except ValueError as error:
        logger.warning(
            'Unable to analyze ticker for pair %s: %s',
            pair,
            str(error)
        )
        return None
    except Exception as error:
        logger.exception(
            'Unexpected error when analyzing ticker for pair %s: %s',
            pair,
            str(error)
        )
        return None
    
    return dataframe


dfs = {}
for t in timeframes:
    dfs[t] = get_df(pair=pair, interval=t)


# whitelist = copy.deepcopy(config['exchange']['pair_whitelist'])

# for _pair in whitelist:
#     df = get_df(pair=_pair, interval=interval)
# #     print (df)
#     pivots = peak_valley_pivots(df.close.values, 0.02, -0.01)
#     print (pivots)
#     plot_pivots(df.close, pivots)
#     df[pivots] = pivots
#     gentrends(df)
#     print (df.head())



# pair_obj = Pair.query.filter(Pair.pair.is_(pair)).first()

# if pair_obj == None:
#     pair_obj = Pair(
#         pair=pair
#     )
#     Pair.session.add(pair_obj)
    
#     # trend = get_trend(pair, 30)
#     # # Not in downtrend in the last 24hrs
#     # if trend != 1:
#     #     sanitized_whitelist.remove(pair)
#     Pair.session.flush()



# def get_tests(df, trend_name):
#     t_r = df.loc[in_range(df['high'],df[trend_name], 0.00001) | in_range(df['low'],df[trend_name], 0.00001)]
#     return t_r


2018-05-06 15:48:18,123 - freqtrade.configuration - INFO - Using config: ../config_testing.json ...
2018-05-06 15:48:18,125 - freqtrade.configuration - INFO - Validating configuration ...
2018-05-06 15:48:18,131 - freqtrade.configuration - INFO - Log level set at 20
2018-05-06 15:48:18,134 - freqtrade.configuration - INFO - Using max_open_trades: 50 ...
2018-05-06 15:48:18,138 - freqtrade.configuration - INFO - Parameter --datadir detected: freqtrade/tests/testdata ...
2018-05-06 15:48:18,141 - freqtrade.strategy.strategy - INFO - Load strategy class: DefaultStrategy (user_data.strategies.trend001.py)
2018-05-06 15:48:18,153 - freqtrade.exchange - INFO - Instance is running with dry_run enabled
2018-05-06 15:48:18,155 - freqtrade.exchange - INFO - Using Exchange "Bittrex"
2018-05-06 15:48:18,163 - requests.packages.urllib3.connectionpool - INFO - Starting new HTTPS connection (1): bittrex.com


interval :  1d


2018-05-06 15:48:19,843 - __main__ - WARNING - Unable to analyze ticker for pair BTC/USDT: window must be an integer


interval :  1h


2018-05-06 15:48:29,860 - freqtrade.exchange - WARNING - <function get_ticker_history at 0x7f8fa8c37268> returned exception: "Could not load ticker history due to networking error. Message: bittrex GET https://bittrex.com/api/v2.0/pub/market/GetTicks?tickInterval=hour&marketName=USDT-BTC HTTPSConnectionPool(host='bittrex.com', port=443): Read timed out. (read timeout=10)"
2018-05-06 15:48:29,861 - freqtrade.exchange - WARNING - retrying <function get_ticker_history at 0x7f8fa8c37268> still for 3 times
2018-05-06 15:48:29,867 - requests.packages.urllib3.connectionpool - INFO - Starting new HTTPS connection (2): bittrex.com


1440
44
42


2018-05-06 15:48:34,689 - __main__ - WARNING - Unable to analyze ticker for pair BTC/USDT: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()


               name  tests
127   trend_648-942     17
135   trend_701-745     16
204   trend_261-485     14
203   trend_261-466     14
136   trend_701-761     14
111   trend_541-942     13
126   trend_648-885     13
208   trend_286-377     13
45   trend_328-1064     12
20   trend_134-1064     12
91   trend_477-1064     12
84   trend_432-1064     12
28   trend_150-1064     12
110   trend_541-885     12
92   trend_477-1096     11
104  trend_495-1064     11
105  trend_495-1096     11
69   trend_405-1064     11
54   trend_343-1064     11
245   trend_852-984     11
interval :  1m
13932
285
284


2018-05-06 15:55:20,031 - __main__ - WARNING - Unable to analyze ticker for pair BTC/USDT: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()


                 name  tests
1656  trend_6866-7496    232
1593  trend_5977-6241    230
1655  trend_6866-7394    225
1666  trend_7009-7394    222
1667  trend_7009-7496    218
48     trend_980-2316    216
1675  trend_7098-7394    213
1217    trend_666-769    213
1654  trend_6866-7009    207
113   trend_2019-2038    206
1622  trend_6680-7839    206
1641  trend_6813-7839    204
1703  trend_7496-7839    202
1711  trend_7645-7812    201
54    trend_1036-2224    201
127   trend_2445-2818    199
1640  trend_6813-7812    198
7       trend_389-798    198
1621  trend_6680-7812    197
1639  trend_6813-7496    196


In [2]:


def get_tests(df, trend_name, min_tests):
    trend = df[trend_name]
    tolerance = 0.01
    t_r = df.loc[(in_range(df['high'],trend, tolerance) 
                  | in_range(df['low'],trend, tolerance)
                  | in_range(df['close'],trend, tolerance)
                  | in_range(df['low'],trend, tolerance))
                  & df['pivots']!=0]

    trend_tests = len(t_r)
#     trends_strength['trend'].append(trend)
#     print(trend_name)
    trends_strength['name'].append(trend_name)
    trends_strength['tests'].append(trend_tests)
    return t_r

def gentrends(df, charts=True, pair='default_filename_plot'):

    h = df.loc[df['pivots']==1]
    l = df.loc[df['pivots']==-1]
    
    print (len(df))
    print (len(h))
    print (len(l))
    

    trend_h = []
    global trends_strength
    trends_strength = {'name':[], 'tests':[]}

    tests = 5
    for i in range(0, len(h) - 1):

        ax = h.index[i]
        ay = h.iloc[i].high
#         print ('a', ax, ay)
        bx = h.index[i+1]
        by = h.iloc[i+1].high
        t = df.index[ax:]
        trend_name = 'trend_'+str(ax)+'-'+str(bx)

        slope, intercept, r_value, p_value, std_err = stats.linregress([ax, bx], [ay, by])
        trend_h = polyval([slope,intercept],t)
        df.loc[h.index[i]:,trend_name] = trend_h
#         print (df)
        
        t_r = get_tests(df, trend_name, tests)
        


#         next_waves = df.loc[df['pivots']==1][i+1:]
        next_waves = h[i+1:]
#         print ('point A: ', ax, 'c: ', ay, '-------------------------------------------------')
        for ib in range(0, len(next_waves)):
            bx = next_waves.index[ib]
            by = next_waves.iloc[ib].high

#             print(df[df.high > df ])

#             print ('\t\tpoint B: ', bx, 'c: ', next_waves.iloc[ib].high, 'trend_point: ', df.loc[bx][trend_name])

            if by > df.loc[bx][trend_name]:
                t = df.index[h.index[i]:]
                slope, intercept, r_value, p_value, std_err = stats.linregress([ax, bx], [ay, by])
                trend_next_wave = polyval([slope,intercept],t)
                trend_name = 'trend_'+str(ax)+'-'+str(bx)
                df.loc[h.index[i]:,trend_name] = trend_next_wave
#                 plt.plot(t, trend_next_wave, 'r', label=trend_name, alpha=0.3)
                t_r = get_tests(df, trend_name, tests)



        
    tests = 20
    for i in range(0, len(l) - 1):

        ax = l.index[i]
        ay = l.iloc[i].low
#         print ('a', ax, ay)
        bx = l.index[i+1]
        by = l.iloc[i+1].low
        t = df.index[ax:]
        trend_name = 'trend_'+str(ax)+'-'+str(bx)

        slope, intercept, r_value, p_value, std_err = stats.linregress([ax, bx], [ay, by])
        trend = polyval([slope,intercept],t)
#         plt.plot(t, trend, 'g', label='fitted line', alpha=0.5)
        df.loc[l.index[i]:,trend_name] = trend

#         plt.scatter(ax, ay, color='r')
#         plt.scatter(bx, by, color='r')

        t_r = get_tests(df, trend_name, tests)
        

#         if len(t_r)-1 > tests:
#             plt.plot(t, trend, 'g', label=trend_name)
#             for it in range(0, len(t_r)):
#                 plt.scatter(t_r.index[it], t_r.iloc[it].low, color='c')
                
        next_waves = l[i+1:]
        for ib in range(0, len(next_waves)):
            bx = next_waves.index[ib]
            by = next_waves.iloc[ib].low
            if next_waves.iloc[ib].low < df.loc[bx][trend_name]:
                t = df.index[l.index[i]:]
                slope, intercept, r_value, p_value, std_err = stats.linregress([ax, bx], [ay, by])
                trend_next_wave = polyval([slope,intercept],t)
                trend_name = 'trend_'+str(ax)+'-'+str(bx)
                df.loc[l.index[i]:,trend_name] = trend_next_wave
                t_r = get_tests(df, trend_name, tests)
#                 plt.plot(t, trend_next_wave, 'g', label='fitted line', alpha=0.3)

    trends = DataFrame(data=trends_strength).sort_values('tests', ascending=False)
#     print (trends)
#     above_mean = trends.loc[trends.tests > trends.tests.mean()][:10]
#     print(trends.tests)
    above_mean = trends.loc[trends.tests > trends.tests.mean()][:20]
    print (above_mean)
    max_tests = above_mean.tests.max()
    for i, row in above_mean.iterrows():
#         print (row)
#         print('row name ', row.tests)
        plt.plot(df.index, df[row['name']], 'grey', label=trend_name)
        
    #     print (len(t_r))
#     for t in len(0, len(trends)):
#         trend_name = trends.index[t]['name']
#         plt.plot(df.index, df[trend_name], 'r', label=trend_name)
# #         for it in range(0, trend_tests):
# #             plt.scatter(t_r.index[it], t_r.iloc[it].close, color='c')

#     df['s1'] = df.loc[df.filter(regex='trend-$', axis=1)[]
    return df

from math import sqrt


#     timeframe_volat = {
#                     '1d':0.02,
#                     '1h':0.015,
#                     '5m':0.008,
#                     '1m':0.005}

timeframe_volat = {
                '1d':1.1,
                '1h':0.0009,
                '5m':4,
                '1m':4}

volat_window = {
                '1d':0.5,
                '1h':2,
                '5m':10,
                '1m':30}
# Plots

def plot_pivots(X, L, H, pivots):
    plt.plot(np.arange(len(X)), X, 'k:', alpha=0.2)
    plt.plot(np.arange(len(X))[pivots != 0], X[pivots != 0], 'k-')
    plt.scatter(np.arange(len(X))[pivots == 1], H[pivots == 1], color='r', alpha=0.3)
    plt.scatter(np.arange(len(X))[pivots == -1], L[pivots == -1], color='g', alpha=0.3)
#     plt.show()
    pass

for t in timeframes:

    df_long = dfs[t]
    timeframe = 2000
    df = df_long
#     volat = sqrt(df['stddev'].mean()) * timeframe_volat[t]
#     print (volat)
#     volat = timeframe_volat[t]
#     df['bb_exp'] = (df.bb_upperband - df.bb_lowerband) / df.bb_lowerband
    window = volat_window[t]
    df['bb_exp'] = (df.bb_upperband.rolling(window=window).max() - df.bb_lowerband.rolling(window=window).min()) / df.bb_upperband.rolling(window=window).max()
    pivots = peak_valley_pivots(df.low.values, df.high.values, df.bb_exp.values)
    df['pivots'] = np.transpose(np.array((pivots)))

#     df['pct_change'] = df.close.pct_change()
#     df['bb_upperband'] = np.log(df.bb_upperband)
#     df['bb_middleband'] = np.log(df.bb_middleband)
#     df['bb_lowerband'] = np.log(df.bb_lowerband)
    
#     print (df.bb_exp)

#     print(df)
    # eje y en log scale

        
#     plt.yscale('log')
    

    
    df['high'] = np.log(df.high)
    df['close'] = np.log(df.close)
    df['open'] = np.log(df.open)
    df['low'] = np.log(df.low)
    
    plt.figure(num=0, figsize=(50,30))
#     plt.xlim(0, len(df.close))
#     plt.ylim(df.low.min()*0.99, df.high.max()*1.01)
    
    
    gentrends(df)
    plt.plot(df.high, 'r', alpha=0.5)
    plt.plot(df.close, 'k', alpha=0.5)
    plt.plot(df.low, 'g', alpha=0.5)

    plot_pivots(df.close.values, df.low.values, df.high.values, pivots)
    legend(['pivots','trend', 'close'])
    plt.show()

AttributeError: 'NoneType' object has no attribute 'bb_upperband'